# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
import gluonts
gluonts.__version__

'0.4.2'

In [2]:
import mxnet
mxnet.__version__

'1.4.1'

In [3]:
# !pip install gluonts

In [4]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [11]:
def deepar(data="m4_quarterly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
#     predictor = estimator.train(training_data=dataset.train, 
#                                 validation_data=dataset.test)
    
    predictor = estimator.train(training_data=dataset.train)
    
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    
    return output 


In [15]:
#%%time
res = deepar(data="m4_yearly", seed=42, epochs=100, batches=100)
pprint(res)
pd.DataFrame([res])

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_yearly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.12it/s, avg_epoch_loss=8.14]
INFO:root:Epoch[0] Elapsed time 7.627 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.140023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.14it/s, avg_epoch_loss=6.82]
INFO:root:Epoch[1] Elapsed time 7.617 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.816145
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.53it/s, avg_epoch_loss=6.89]
INFO:root:Epoch[2] Elapsed time 7.402 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.890125
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 16.86it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[3] Elapsed time 5.947 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.256

100%|██████████| 100/100 [00:07<00:00, 13.79it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[36] Elapsed time 7.258 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.449463
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 16.80it/s, avg_epoch_loss=5.56]
INFO:root:Epoch[37] Elapsed time 5.957 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.560893
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 15.07it/s, avg_epoch_loss=6.2]
INFO:root:Epoch[38] Elapsed time 6.643 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.198479
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.83it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[39] Elapsed time 7.238 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.524188
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 15.28it/s, avg_epoch_loss=7]
INFO:root:Epoch[40] Elapsed time 6.553 seconds
IN

{'MASE': 3.47222148,
 'MSIS': 66.72490916,
 'epochs': 100,
 'sMAPE': 0.14227351,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.13633944,
 'wQuantileLoss[0.9]': 0.0901874}


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9],epochs,seed
0,3.472221,0.142274,66.724909,0.136339,0.090187,100,42


In [ ]:
#%%time
res = deepar(data="m4_yearly", seed=42, epochs=250, batches=100)
pprint(res)
pd.DataFrame([res])

In [ ]:
#%%time
res = deepar(data="m4_yearly", seed=43, epochs=250, batches=100)
pprint(res)
pd.DataFrame([res])

In [ ]:
#%%time
res = deepar(data="m4_yearly", seed=44, epochs=250, batches=100)
pprint(res)
pd.DataFrame([res])

In [ ]:
!jupyter nbconvert --output-dir="./html_outputs" --to html "DeepAR 09 - DeepAR experiments with gluonts".ipynb

In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results